In [15]:
# pytorch提供了某些领域特定的库，Text Vision Audio 其中都包含数据集
# 本教程介绍如何使用一个Vision的数据集
# datasets模块包含现实世界视觉数据的数据集 (https://pytorch.org/vision/stable/datasets.html?spm=a2c6h.12873639.article-detail.34.423a18baLTdMlS)
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# 从数据集中下载训练数据
training_data = datasets.FashionMNIST(
    root="../data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 从数据集中下载测试数据
test_data = datasets.FashionMNIST(
    root="../data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# 将上面dataset传递给loader,定义一个批量大小为64，表示loader迭代对象中每个元素并返回一个包含64个特征和标签的批次。
batch_size = 64
train_dataloader = DataLoader(training_data,batch_size = batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)
for x,y in test_dataloader:
    print(f"Shape of X [N,C,H,W]:{x.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


#  创建模型
# 设置设备
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backend.mps.isavailable()
    else "cpu"
)
print(f"Using {device} device")
# 定义模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

# 优化神经网络
# 定义损失函数和优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

# 迭代训练
def train(dataloader,model,loss_fn,optimizer):
    # 总样本数
    size = len(dataloader.dataset)
    model.train()
    for batch,(X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)

        # 计算训练误差
        pred = model(X)
        loss = loss_fn(pred,y)

        # 反向传播
        loss.backward()
        # 更新权重
        optimizer.step()
        # 清空梯度
        optimizer.zero_grad()

        # 每100个批次索引，就输出损失值和处理过的数据量
        if batch % 100 == 0:
            # item()将张量类型转换为python数字便于观察
            loss,current = loss.item(), (batch + 1) * len(X)
            # 格式化为7位浮点数和5位整数
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss,correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X, y = X.to(device),y.to(device)
            pred = model(X)
            test_loss +=loss_fn(pred,y).item()
            correct+=(pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Shape of X [N,C,H,W]:torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.304386  [   64/60000]
loss: 2.295717  [ 6464/60000]
loss: 2.277312  [12864/60000]
loss: 2.269320  [19264/60000]
loss: 2.260326  [25664/60000]
loss: 2.223692  [32064/60000]
loss: 2.231730  [38464/60000]
loss: 2.204584  [44864/60000]
loss: 2.195259  [51264/60000]
loss: 2.161592  [57664/60000]
Test Error: 
 Accuracy: 40.1%, Avg loss: 2.160934 

Epoch 2
-------------------------------
loss: 2.173401  [   64/60000]
loss: 2.164267  [ 6464/60000]
loss: 2.106642  [12864/60000]
loss: 2.117654  [19264/60000]
loss: 2.070904  [2566

In [18]:
# 模型保存
torch.save(model.state_dict(),"../data/model001.pth")
print("Saved PyTorch Model State to model001.pth")

Saved PyTorch Model State to model001.pth


In [19]:
# 加载模型
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("../data/model001.pth"))
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [24]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
